In [1]:
import numpy as np
import cmath
import pyperclip
import geogebra
import limb
import movement_assignment
import checks
import basis

In [2]:
class platform:
    def __init__(self,d,D,l1,l2,max_limb_error,max_limb_try_count,max_joint_angle):
        "d: distancia de cada junta universal en la plataforma al centro"
        
        "D: distancia del eje de cada motor al centro de la base"
        
        "la altura del eje del motor es conciderada el plano z=0"
        
        "l1: long del antebrazo"
        
        "l2: long del brazo"
        
        "max_limb_error: máximo error (en las mismas unidades de l1 y l2) que es permitido para posicionar el brazo"
        
        "max_limb_try_count: máxima cantidad de iteraciones para calcular el ángulo de un motor, si la cantidad"
        "Si la cantidad requerida de iteraciones es mayor, genera un error llamado 'Point unreachable' "
        
        "max_joint_angle:angulo máximo que puede tener la junta universal, una posición que lo exceda genera un error"
        "llamado 'Joint angle exceeded'"
        
        self.c1=np.array([0,-d,0])
        self.c2=np.array([-d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        self.c3=np.array([d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        
        self.d=d
        self.D=D

        self.l1=l1
        self.l2=l2
        
        self.max_limb_error=max_limb_error
        self.max_limb_try_count=max_limb_try_count
        
        #defino los angulos de los motores
        self.ang1=0
        self.ang2=0
        self.ang3=0
        #defino los angulos máximos que se les permite rotar a los motores
        self.angle_max=90
        self.angle_min=-90
        #defino el ángulo máximo permitido en las juntas universales
        self.max_joint_angle=max_joint_angle

          
    def rotate(self,movement,joint,angle):
        #Cambio la base de referencia a la de la junta en la cual se realiza el movimiento
        print("joint={}".format(joint))
        if joint=="C1":
            self.c1=basis.base_change_cannon_to_m1(self.c1)
            self.c2=basis.base_change_cannon_to_m1(self.c2)
            self.c3=basis.base_change_cannon_to_m1(self.c3)              
        if joint=="C2":
            self.c1=basis.base_change_cannon_to_m2(self.c1)
            self.c2=basis.base_change_cannon_to_m2(self.c2)
            self.c3=basis.base_change_cannon_to_m2(self.c3)                 
        if joint=="C3":
            self.c1=basis.base_change_cannon_to_m3(self.c1)
            self.c2=basis.base_change_cannon_to_m3(self.c2)
            self.c3=basis.base_change_cannon_to_m3(self.c3)
        print("C1={}".format(self.c1))
        print("C2={}".format(self.c2))
        print("C3={}".format(self.c3))
        #Realizo la rotación
        if movement=="pitch":
            self.c1=basis.rotate_in_x_axis(self.c1,angle)
            self.c2=basis.rotate_in_x_axis(self.c2,angle)
            self.c3=basis.rotate_in_x_axis(self.c3,angle)    
            print("pitching!!")
        if movement=="roll":
            print("rolling!!")
            self.c1=basis.rotate_in_y_axis(self.c1,angle)
            self.c2=basis.rotate_in_y_axis(self.c2,angle)
            self.c3=basis.rotate_in_y_axis(self.c3,angle)
        print("C1={}".format(self.c1))
        print("C2={}".format(self.c2))
        print("C3={}".format(self.c3))
        #Corrijo el offset de la plataforma
        ##Siempre tomo la junta de la izquiera como referencia en cada caso
        if joint=="C1":
            self.apply_offset(self.c2)
        if joint=="C2":
            self.apply_offset(self.c3)
        if joint=="C3":
            self.apply_offset(self.c1)
        #Cambiamos a la base orginal
        if joint=="C1":
            self.c1=basis.base_change_m1_to_cannon(self.c1)
            self.c2=basis.base_change_m1_to_cannon(self.c2)
            self.c3=basis.base_change_m1_to_cannon(self.c3)              
        if joint=="C2":
            self.c1=basis.base_change_m2_to_cannon(self.c1)
            self.c2=basis.base_change_m2_to_cannon(self.c2)
            self.c3=basis.base_change_m2_to_cannon(self.c3)                 
        if joint=="C3":
            self.c1=basis.base_change_m3_to_cannon(self.c1)
            self.c2=basis.base_change_m3_to_cannon(self.c2)
            self.c3=basis.base_change_m3_to_cannon(self.c3)           
        
    def apply_offset (self,pos):
        "la función desplaza la plataforma el offset correspondiente para compenzar el introducido por la rotación"
        allowed_pos=-pos[0]*np.tan(np.pi/6)
        offset=np.array([0,allowed_pos-pos[1],0])
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        
        
    def calculate_P(self):
        #Función que obtiene las coordenadas de las juntas c1,c2,c3 en las bases de los motores m1,m2 y m3
        #respectivamente. Se le suma el vector (0,D,0) para establecer el punto de origen en el rotor del motor
        P1=basis.base_change_cannon_to_m1(self.c1)+np.array([0,self.D,0])
        P2=basis.base_change_cannon_to_m2(self.c2)+np.array([0,self.D,0])
        P3=basis.base_change_cannon_to_m2(self.c3)+np.array([0,self.D,0])
        #elimino la coordenana en x. Dado que las juntas se desplazan por el plano y-z de cada motor la coordenada en x es siempre cero
        P1=np.array([P1[1],P1[2]])
        P2=np.array([P2[1],P2[2]])
        P3=np.array([P3[1],P3[2]])
        return P1,P2,P3
    

        
    def correct_height(self,h):
        c1x=self.c1[0]
        c2x=self.c2[0]
        c3x=self.c3[0]
        
        c1y=self.c1[1]
        c2y=self.c2[1]
        c3y=self.c3[1]
        
        c1z=self.c1[2]
        c2z=self.c2[2]
        c3z=self.c3[2]
        
        A=np.array([[c1x,c1y,1],[c2x,c2y,1],[c3x,c3y,1]])
        B=np.array([c1z,c2z,c3z]).transpose()
        params=np.linalg.solve(A,B)
        offset=np.array([0,0,h-params[2]])
        #corrijo la altura
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        

    def solve_platform(self,r_x,r_y,h):
        if r_x==0 and r_y==0:
            #Este if se utiliza para no calcular rotaciones cuando los ángulos de ambos ejes son cero
            self.correct_height(h)
        else:
            joint,movement,angle=movement_assignment.assign_movement(r_x,r_y)
            #print("movement={}".format(movement))
            #print("joint={}".format(joint))
            #print("angle={}".format(angle))
            self.rotate(movement,joint,angle)
            self.correct_height(h)
        
        P1,P2,P3=self.calculate_P()
        limb_i=limb.limb(self.l1,self.l2)
        self.ang1=limb_i.calculate_motor_angle(P1,self.max_limb_error,self.max_limb_try_count)
        self.ang2=limb_i.calculate_motor_angle(P2,self.max_limb_error,self.max_limb_try_count)
        self.ang3=limb_i.calculate_motor_angle(P3,self.max_limb_error,self.max_limb_try_count)
        #Chekeo el ángulo necesario en las juntas universales
        for joint in ["C1","C2","C3"]:
            angle=checks.get_joint_angle(self.c1,self.c2,self.c3,joint)
            if np.abs(angle)> self.max_joint_angle:
                #raise ValueError('Joint angle exceeded\n {}.angle={}º >{}º'.format(joint,angle*180/np.pi,self.max_joint_angle*180/np.pi))
                print('Joint angle exceeded\n {}.angle={}º >{}º'.format(joint,angle*180/np.pi,self.max_joint_angle*180/np.pi))
                return -1
            else:
                return 0       

In [3]:
angle_spacex=np.linspace(start=-19*np.pi/180,stop=19*np.pi/180,num=10)
angle_spacey=np.linspace(start=-19*np.pi/180,stop=19*np.pi/180,num=10)
h_space=np.linspace(start=7,stop=7,num=1)
s1=geogebra.geogebra_script()
point=0
for height in [10,11,12,13,14]:
    for angle_x in angle_spacex:
        for angle_y in angle_spacey:
            p1=platform(d=6,D=10,l1=6,l2=12,max_limb_error=0.00001,max_limb_try_count=30,max_joint_angle=23*np.pi/180)
            result=p1.solve_platform(r_x=angle_x,r_y=angle_y,h=height) 
            if result==-1:
                print("angle exceeded")
                print("angle_x={}".format(angle_x*180/np.pi))
                print("angle_y={}".format(angle_y*180/np.pi))
            else:
                check_h=checks.check_motor_height (p1.c1,p1.c2,p1.c3,p1.D)
                if check_h==-1:
                    print("Motor choca con el plato:\n")
                    "[BUG]:si el motor choca contra el plato y al mismo tiempo el punto es unreachable no se puede generar el"
                    "Script de geogebra, porque no hay angulos para armar los limbs"
                    #p1.generate_geogebra()
                P1,P2,P3=p1.calculate_P()
                name="P"+str(point)
                s1.add_2D_point(P1,name)
                point=point+1

joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.69721032  1.31341406]
C2=[5.19615242 2.69721032 1.31341406]
C3=[ 1.53837015e-15 -5.39442063e+00 -2.62682813e+00]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.75223554  1.1938172 ]
C2=[5.19615242 2.75223554 1.1938172 ]
C3=[ 1.53837015e-15 -5.50447109e+00 -2.38763440e+00]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.7935275   1.09371117]
C2=[5.19615242 2.7935275  1.09371117]
C3=[ 1.53837015e-15 -5.58705500e+00 -2.18742233e+00]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitchin

joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
rolling!!
C1=[ 5.16097737  3.         -0.60358311]
C2=[-1.52795624e-15 -6.00000000e+00  1.78696497e-16]
C3=[-5.16097737  3.          0.60358311]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.99593175  0.15618236]
C2=[5.19615242 2.99593175 0.15618236]
C3=[ 1.53837015e-15 -5.99186351e+00 -3.12364712e-01]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rolling!!
C1=[-5.18910601  3.          0.27051578]
C2=[ 5.18910601  3.         -0.27051578]
C3=[ 1.53628399e-15 -6.00000000e+00 -8.00887582e-17]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rol

joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
rolling!!
C1=[ 4.83853156  3.         -1.89436331]
C2=[-1.43249310e-15 -6.00000000e+00  5.60844204e-16]
C3=[-4.83853156  3.          1.89436331]
Point unreachable
P=(12.793527498057509,12.894363310663948)
joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
rolling!!
C1=[ 4.9446549   3.         -1.59699341]
C2=[-1.46391192e-15 -6.00000000e+00  4.72805028e-16]
C3=[-4.9446549   3.          1.59699341]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.90095947  0.76448292]
C2=[5.19615242 2.90095947 0.76448292]
C3=[ 1.53837015e-15 -5.80191895e+00 -1.52896583e+00]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.      

joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
pitching!!
C1=[ 0.         -5.79785261 -1.54431381]
C2=[-5.19615242  2.8989263   0.7721569 ]
C3=[5.19615242 2.8989263  0.7721569 ]
joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
pitching!!
C1=[ 0.         -5.6696687  -1.96337894]
C2=[-5.19615242  2.83483435  0.98168947]
C3=[5.19615242 2.83483435 0.98168947]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rolling!!
C1=[-4.8862262   3.         -1.76770854]
C2=[4.8862262  3.         1.76770854]
C3=[ 1.44661356e-15 -6.00000000e+00  5.23346860e-16]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rolling!!
C1=[-4.99559064  3.         -1.42971121]
C2=[4.99559064 3.         1.42971121]
C3=[ 1.47899193e-15 -

joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.82106401  1.020587  ]
C2=[5.19615242 2.82106401 1.020587  ]
C3=[ 1.53837015e-15 -5.64212803e+00 -2.04117400e+00]
joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
rolling!!
C1=[ 0. -6.  0.]
C2=[-4.91007712  3.         -1.70033604]
C3=[4.91007712 3.         1.70033604]
Point unreachable
P=(12.834834349395106,13.700336035749666)
joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
rolling!!
C1=[ 0. -6.  0.]
C2=[-4.91007712  3.         -1.70033604]
C3=[4.91007712 3.         1.70033604]
Point unreachable
P=(12.834834349395106,13.700336035749666)
joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
pitching!!
C1=[ 5.19615242  2.82106401 

...

joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.9318326   0.63589121]
C2=[5.19615242 2.9318326  0.63589121]
C3=[ 1.53837015e-15 -5.86366519e+00 -1.27178242e+00]
joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
rolling!!
C1=[ 0. -6.  0.]
C2=[-5.10487093  3.         -0.96968695]
C3=[5.10487093 3.         0.96968695]
Point unreachable
P=(12.94729860645822,13.969686945930633)
joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
rolling!!
C1=[ 0. -6.  0.]
C2=[-5.10487093  3.         -0.96968695]
C3=[5.10487093 3.         0.96968695]
Point unreachable
P=(12.94729860645822,13.969686945930633)
joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
pitching!!
C1=[ 5.19615242  2.9318326  -0

P=(13.0,14.03888553646427)
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rolling!!
C1=[-4.8862262   3.         -1.76770854]
C2=[4.8862262  3.         1.76770854]
C3=[ 1.44661356e-15 -6.00000000e+00  5.23346860e-16]
Point unreachable
P=(13.089467993005114,13.0)
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rolling!!
C1=[-4.99559064  3.         -1.42971121]
C2=[4.99559064 3.         1.42971121]
C3=[ 1.47899193e-15 -6.00000000e+00  4.23279548e-16]
Point unreachable
P=(13.057897199261468,13.0)
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rolling!!
C1=[-5.07808301  3.         -1.10139589]
C2=[5.07808301 3.         1.10139589]
C3=[ 1.50341458e-15 -6.00000000e+00  3.26078686e-16]
Point unreachable
P=(13.03408370

Point unreachable
P=(13.0,14.085321350840589)
joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
rolling!!
C1=[ 0. -6.  0.]
C2=[-4.91007712  3.          1.70033604]
C3=[ 4.91007712  3.         -1.70033604]
joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
rolling!!
C1=[ 0. -6.  0.]
C2=[-4.91007712  3.          1.70033604]
C3=[ 4.91007712  3.         -1.70033604]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.82106401 -1.020587  ]
C2=[ 5.19615242  2.82106401 -1.020587  ]
C3=[ 1.53837015e-15 -5.64212803e+00  2.04117400e+00]
Point unreachable
P=(12.731596020984657,14.976439655630562)
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
pitching!!
C1=[-5.19615242  2.7935275  -1.09

joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rolling!!
C1=[-4.83853156  3.          1.89436331]
C2=[ 4.83853156  3.         -1.89436331]
C3=[ 1.43249310e-15 -6.00000000e+00 -5.60844204e-16]
Point unreachable
P=(13.103236250971245,14.0)
joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
rolling!!
C1=[ 4.8862262   3.         -1.76770854]
C2=[-1.44661356e-15 -6.00000000e+00  5.23346860e-16]
C3=[-4.8862262   3.          1.76770854]
Point unreachable
P=(12.821064013989773,15.767708541280419)
joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
rolling!!
C1=[ 4.99559064  3.         -1.42971121]
C2=[-1.47899193e-15 -6.00000000e+00  4.23279548e-16]
C3=[-4.99559064  3.          1.42971121]
Point unreachable
P=(12.884205601477

Point unreachable
P=(12.931832595155006,12.898604112065062)
joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
rolling!!
C1=[4.99559064 3.         1.42971121]
C2=[-1.47899193e-15 -6.00000000e+00 -4.23279548e-16]
C3=[-4.99559064  3.         -1.42971121]
Point unreachable
P=(12.88420560147706,12.570288789571457)
joint=C2
C1=[5.19615242 3.         0.        ]
C2=[-1.53837015e-15 -6.00000000e+00  0.00000000e+00]
C3=[-5.19615242  3.          0.        ]
rolling!!
C1=[4.8862262  3.         1.76770854]
C2=[-1.44661356e-15 -6.00000000e+00 -5.23346860e-16]
C3=[-4.8862262   3.         -1.76770854]
joint=C3
C1=[-5.19615242  3.          0.        ]
C2=[5.19615242 3.         0.        ]
C3=[ 1.53837015e-15 -6.00000000e+00  0.00000000e+00]
rolling!!
C1=[-4.83853156  3.         -1.89436331]
C2=[4.83853156 3.         1.89436331]
C3=[ 1.43249310e-15 -6.00000000e+00  5.60844204e-16]
Point unreachable
P=(13.10323625

In [4]:
pyperclip.copy(s1.print_script())

Execute[{
"P0=(4.000,11.461)"
,"P1=(4.000,11.301)"
,"P2=(4.000,11.175)"
,"P3=(4.000,11.085)"
,"P4=(3.835,10.000)"
,"P5=(3.835,10.000)"
,"P6=(4.000,8.915)"
,"P7=(4.000,8.825)"
,"P8=(4.000,8.699)"
,"P9=(4.606,12.275)"
,"P10=(4.496,7.932)"
,"P11=(4.000,11.119)"
,"P12=(4.000,10.969)"
,"P13=(4.000,10.859)"
,"P14=(3.899,10.000)"
,"P15=(3.899,10.000)"
,"P16=(4.000,9.141)"
,"P17=(4.000,9.031)"
,"P18=(4.378,11.816)"
,"P19=(4.496,12.068)"
,"P20=(4.413,8.106)"
,"P21=(4.290,8.403)"
,"P22=(4.000,10.791)"
,"P23=(4.000,10.651)"
,"P24=(3.947,10.000)"
,"P25=(3.947,10.000)"
,"P26=(4.000,9.349)"
,"P27=(4.198,11.324)"
,"P28=(4.290,11.597)"
,"P29=(4.413,11.894)"
,"P30=(4.358,8.232)"
,"P31=(4.232,8.570)"
,"P32=(4.136,8.899)"
,"P33=(4.000,10.471)"
,"P34=(4.000,10.351)"
,"P35=(4.000,9.649)"
,"P36=(4.073,10.806)"
,"P37=(4.136,11.101)"
,"P38=(4.232,11.430)"
,"P39=(4.358,11.768)"
,"P40=(4.495,11.906)"
,"P41=(4.303,11.517)"
,"P42=(4.158,11.110)"
,"P43=(4.041,9.396)"
,"P44=(4.000,10.156)"
,"P45=(4.008,10.271)"
,"P

In [8]:
p1=platform(d=6,D=10,l1=6,l2=12,max_limb_error=0.00001,max_limb_try_count=30,max_joint_angle=23*np.pi/180)
p1.solve_platform(r_x=0*np.pi/180,r_y=-20*np.pi/180,h=10) 
print("C1={}".format(p1.c1))
print("C2={}".format(p1.c2))
print("C3={}".format(p1.c3))

joint=C1
C1=[ 0 -6  0]
C2=[-5.19615242  3.          0.        ]
C3=[5.19615242 3.         0.        ]
pitching!!
C1=[ 0.         -5.63815572  2.05212086]
C2=[-5.19615242  2.81907786 -1.02606043]
C3=[ 5.19615242  2.81907786 -1.02606043]
C1=[ 0.         -5.45723359 11.98627059]
C2=[-5.19615242  3.          8.9080893 ]
C3=[5.19615242 3.         8.9080893 ]


In [9]:
geogebra_script=geogebra.geogebra_script()
pyperclip.copy(geogebra_script.generate_geogebra(p1))

Execute[{
"C1=(0.000,-5.457,11.986)"
,"C2=(-5.196,3.000,8.908)"
,"C3=(5.196,3.000,8.908)"
,"Polygon(C1,C2, C3)"
,"M1=(0.000,-10.000,0.000)"
,"M2=(-8.660,5.000,0.000)"
,"M3=(8.660,5.000,0.000)"
,"AUX0=(0.000,0.000,0.000)"
,"AUX4=(0.000,0.000,1.000)"
,"Plane(M1,AUX0, AUX4)"
,"Plane(M2,AUX0, AUX4)"
,"Plane(M3,AUX0, AUX4)"
,"A1=(0.000,-4.000,0.075)"
,"Segment(M1,A1)"
,"Segment(A1,C1)"
,"A2=(-4.177,2.412,-3.034)"
,"Segment(M2,A2)"
,"Segment(A2,C2)"
,"A3=(3.510,2.027,-0.800)"
,"Segment(M3,A3)"
,"Segment(A3,C3)"
,"AUX1=(0.000,-12.000,2.500)"
,"AUX2=(5.000,-12.000,2.500)"
,"Segment(AUX1,AUX2)"
}]


In [10]:
p1.ang2*180/np.pi

-30.37688955533404